# Pipeline de Transformación de Datos de Encuesta

Este notebook implementa un pipeline para procesar y transformar datos de una encuesta desde un formato ancho (como se exporta típicamente de SPSS) a un formato largo, adecuado para análisis y visualización en herramientas como Power BI.

## 1. Carga de Datos y Limpieza Inicial

El primer paso es cargar los datos de la encuesta desde un archivo `.sav` (SPSS). Utilizamos `pyreadstat` para leer el archivo, ya que nos permite acceder tanto a los datos como a los metadatos (etiquetas de variables y valores).

Realizamos dos acciones iniciales:
1.  Extraemos los metadatos (`dict_value_labels`, `dict_column_labels`) para usarlos más adelante.
2.  Estandarizamos los nombres de las columnas a minúsculas para facilitar el manejo.

In [34]:
import pyreadstat

# Carga el archivo .sav y sus metadatos.
# NOTA: La ruta es local y debe ser ajustada si se ejecuta en otro entorno.
df, meta = pyreadstat.read_sav(
        "/Users/blanc/Library/CloudStorage/GoogleDrive-dblancbellido@gmail.com/.shortcut-targets-by-id/153DRxBD4jspE3sko8JH1R6j2AYv7OrgD/PBI - Comunidad Zona/datos/BASE_PBI.sav",
        apply_value_formats=True,
        formats_as_category=True,
        formats_as_ordered_category=False
)

# Almacena los metadatos en diccionarios para su uso posterior.
dict_value_labels = meta.variable_value_labels
dict_column_labels = meta.column_names_to_labels

# Convierte todos los nombres de columna a minúsculas para consistencia.
df.columns = [col.lower() for col in df.columns]

df.head()

## 2. Clasificación de Variables

Las variables de la encuesta se clasifican en tres grupos para su procesamiento diferenciado:

- **`segmentadores`**: Variables demográficas o de corte que se usarán para segmentar los datos (ej. `sexo`, `edad_tramo`).
- **`simples`**: Preguntas de respuesta única (ej. una escala de satisfacción).
- **`compuestas`**: Preguntas de respuesta múltiple, donde cada opción se representa como una columna separada en el formato original (ej. `transporte1a`, `transporte1b`).

Primero, definimos manualmente la lista de `segmentadores`.

In [35]:
segmentadores = [
    'idbase',
    'sexo',
    'edad_tramo',
    'edad',
    'depto',
    'barrio',
    'hijos',
    'cargo_declarado',
    'industria_rec',
    'modalidad_trabajo',
    'edificios1',
    'cliente_rec',
    'barrio2_monte',
    'canelones',
    'canelones_localidades',
    'canelones_otr',
    'privado_cod',
    'identificador',
    'idequipos',
    'edad_rec',
    'edu_rec',
    'ponderador'
]

A continuación, clasificamos el resto de las columnas como `simples` o `compuestas` de forma programática. La lógica se basa en la nomenclatura de las columnas:

- Si el nombre de la columna contiene un número seguido de una letra (ej. `hijos10a`), se considera `compuesta`.
- De lo contrario, se considera `simple`.

In [36]:
import re

# Función para analizar el nombre de una columna y separarlo en base, número y letra.
def separar_elemento(elemento):
    match = re.search(r'\d', elemento)
    if match:
        pos = match.start()
        parte1 = elemento[:pos]
        parte2 = elemento[pos]
        parte3 = elemento[pos+1:] if pos+1 < len(elemento) else ''
        return parte1, parte2, parte3
    else:
        return elemento, '', ''

# Aplica la función a todas las columnas.
resultado = [separar_elemento(e) for e in df.columns]

# Inicializa las listas, incluyendo 'idbase' y 'ponderador' que son claves para el análisis.
simples = ['idbase', 'ponderador']
compuestas = ['idbase', 'ponderador']

# Itera sobre los nombres de columna para clasificarlos.
for res in resultado:
    variable = ''.join(res)
    if variable not in segmentadores:
        # Si la parte 3 no está vacía, es una variable compuesta (ej. 'hijos1a').
        if res[2] != '':
            compuestas.append(variable)
        else:
            simples.append(variable)


## 3. Creación de DataFrames por Tipo de Variable

Con las variables ya clasificadas, creamos tres DataFrames separados, cada uno conteniendo solo las columnas de su respectivo tipo.

In [37]:
df_segmentadores = df[segmentadores]
df_simples = df[simples]
df_compuestas = df[compuestas]

## 4. Transformación de Formato Ancho a Largo (Pivot)

Ahora, transformamos los DataFrames de variables `simples` y `compuestas` para que tengan un formato largo. Esto se hace con la función `melt` de pandas, que convierte las columnas en filas.

El resultado es un DataFrame con las columnas:
- `idbase`: El identificador del encuestado.
- `ponderador`: El peso de la encuesta.
- `variable`: El nombre de la pregunta (el nombre de la columna original).
- `value`: La respuesta dada.

In [38]:
# Pivotea el DataFrame de variables compuestas.
df_compuestas_pivoted = df_compuestas.melt(
    id_vars = ['idbase', 'ponderador'],
)
df_compuestas_pivoted.dropna(inplace=True)
df_compuestas_pivoted.head()

In [39]:
# Pivotea el DataFrame de variables simples.
df_simples_pivoted = df_simples.melt(
    id_vars = ['idbase', 'ponderador']
)
df_simples_pivoted.dropna(inplace=True)
df_simples_pivoted.head()

## 5. Carga y Procesamiento del Diccionario de Datos

Cargamos un archivo de Excel (`Indice.xlsx`) que funciona como un diccionario de datos. Contiene información adicional sobre cada variable, como el texto completo de la pregunta. 

En este paso, también procesamos la columna `Pregunta` para separar el tema principal de la pregunta específica, creando las columnas `opciones` y `preguntas`.

In [40]:
import pandas as pd
# Carga el archivo Excel que contiene el diccionario de datos.
# NOTA: La ruta es local y debe ser ajustada.
indice = pd.read_excel('/Users/blanc/Library/CloudStorage/GoogleDrive-dblancbellido@gmail.com/.shortcut-targets-by-id/153DRxBD4jspE3sko8JH1R6j2AYv7OrgD/PBI - Comunidad Zona/datos/Indice.xlsx')
# Estandariza la columna 'Indicador' a minúsculas para que coincida con los nombres de las variables.
indice['Indicador'] = indice['Indicador'].str.lower()
# Filtra filas no deseadas.
indice_filtrado = indice[indice['Pregunta'] != 0].copy()
indice_filtrado.head()

In [41]:
# Función para separar el texto de la pregunta en 'opción' y 'pregunta' usando ':' como delimitador.
def pregunta_y_opcion(row):
    pregunta_partida = row.split(':')
    try:
        opcion = pregunta_partida[0]
        pregunta = pregunta_partida[1]
    except:
        opcion = 'No option'
        pregunta = pregunta_partida[0]
    return opcion, pregunta

# Limpia dobles dos puntos para evitar errores en la separación.
indice_filtrado['Pregunta'] = indice_filtrado['Pregunta'].str.replace('::', ':')

# Aplica la función para crear las nuevas columnas.
results = indice_filtrado['Pregunta'].apply(pregunta_y_opcion)

opciones = []
preguntas = []
for row in results:
    opciones.append(row[0])
    preguntas.append(row[1])

indice_filtrado['opciones'] = opciones
indice_filtrado['preguntas'] = preguntas

indice_filtrado.head()

## 6. Enriquecimiento de Datos

Unimos los DataFrames transformados (`simples` y `compuestas`) con el diccionario de datos (`indice_filtrado`). Esto añade el texto de la pregunta y la opción a cada fila, enriqueciendo los datos para el análisis.

In [42]:
# Une el DataFrame de compuestas con el índice.
df_compuestas_joined = df_compuestas_pivoted.merge(
    indice_filtrado[['Indicador', 'preguntas', 'opciones']],
    how = 'inner',
    left_on = 'variable',
    right_on = 'Indicador'
)
df_compuestas_joined.drop(['Indicador'], axis = 1, inplace=True)

df_compuestas_joined.head()

In [43]:
# Une el DataFrame de simples con el índice.
df_simples_joined = df_simples_pivoted.merge(
    indice_filtrado[['Indicador', 'preguntas', 'opciones']],
    how = 'inner',
    left_on = 'variable',
    right_on = 'Indicador'
)
df_simples_joined.drop(['Indicador'], axis = 1, inplace=True)
df_simples_joined.head()

## 7. Unión y Limpieza Final

Combinamos los DataFrames `simples` y `compuestos` ya enriquecidos en un único DataFrame final. Luego, realizamos una limpieza clave: eliminamos las filas con el valor `Unchecked`, que son comunes en preguntas de opción múltiple y no aportan valor al análisis.

Finalmente, exportamos el DataFrame limpio a un archivo CSV.

In [44]:
df_simples_joined.drop_duplicates(inplace=True)

In [45]:
# Concatena los dos DataFrames.
df_union = pd.concat([df_compuestas_joined, df_simples_joined], axis = 0, ignore_index=True)
df_union.drop_duplicates(inplace=True)
df_union.head()

In [46]:
# Filtra las filas que no son 'Unchecked'.
df_pivoted_without_unchecked = df_union[df_union.value != 'Unchecked']
# Exporta el resultado a un archivo CSV.
df_pivoted_without_unchecked.to_csv('df_pivoted_without_unchecked.csv', index=False)

## 8. Procesamiento Adicional de Segmentadores

Los datos de segmentación también se procesan y exportan por separado. Esto incluye:
1.  Crear una columna `Sin filtro` para permitir análisis no segmentados.
2.  Agrupar los departamentos en categorías más amplias (`Montevideo`, `Canelones`, `Resto`).
3.  Pivotar el DataFrame de segmentadores a un formato largo.
4.  Asignar índices numéricos a los valores para su uso en modelos de datos.
5.  Exportar los resultados a archivos CSV.

In [48]:
df_segmentadores['Sin filtro'] = 'Sin filtro'

In [49]:
df_segmentadores['depto'] = df_segmentadores.apply(lambda x: x['depto'] if x['depto'] in ['Montevideo', 'Canelones'] else 'Resto', axis=1)

In [50]:
df_segmentadores_pivoted = df_segmentadores.melt(
    id_vars = ['idbase', 'ponderador'],
)
df_segmentadores_pivoted.dropna(inplace=True)
df_segmentadores_pivoted.head()

In [52]:
alphabet =  {k.lower(): v for k, v in dict_value_labels.items()}
alphabet

In [53]:
def insert_raw_values(row):
    variable = row['variable']
    value = row['value']
    try:
        raw_dict = alphabet[variable]
        raw_value = list(raw_dict.keys())[list(raw_dict.values()).index(value)]
        raw_value_int = int(raw_value)
    except:
        raw_value_int = 0
    return raw_value_int

unique_segmentadores = df_segmentadores_pivoted[['variable','value']].drop_duplicates()

unique_segmentadores['raw_value']= unique_segmentadores.apply(insert_raw_values, axis= 1)

unique_segmentadores_with_index = unique_segmentadores.sort_values(['variable', 'raw_value']).reset_index(drop=True).reset_index()

df_segmentadores_pivoted = df_segmentadores_pivoted.merge(
    unique_segmentadores_with_index,
    left_on = ['variable', 'value'],
    right_on = ['variable', 'value']
)

df_segmentadores_pivoted.to_csv('df_segmentadores_pivoted.csv', index=False)

In [59]:
# Crea una copia para no modificar el DataFrame original.
df_segmentadores_addition = df_segmentadores_pivoted.copy()

# Crea una fila 'Total' para cada grupo de segmentación.
df_segmentadores_addition['value'] = 'Total'
df_segmentadores_addition['index'] = 0

# Concatena el DataFrame original con el de totales.
df_segmentadores_with_totals = pd.concat([df_segmentadores_pivoted, df_segmentadores_addition], axis = 0, ignore_index=True)

# Excluye la categoría 'Sin filtro' que ya cumplió su propósito.
df_segmentadores_with_totals = df_segmentadores_with_totals.copy()[df_segmentadores_with_totals['value'] != 'Sin filtro']

# Exporta el resultado final a CSV.
df_segmentadores_with_totals.to_csv('df_segmentadores_with_totals.csv', index=False)